In [1]:
# Install necessary libraries
# %pip install langchain langchain_community langchain_openai neo4j python-dotenv pypdf tiktoken

import os
import json
import re
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from neo4j import GraphDatabase
import time

In [2]:
# Load environment variables (ensure NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD, OPENAI_API_KEY are set in your .env file)
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
# NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
# NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Embedding model setup
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)
embedding_dimension = 1536 # Dimension for text-embedding-3-small

# Data paths
pdf_path = './dataset/criminal-law.pdf'
precedent_dir = './dataset/precedent_label/' # Directory containing JSON files

In [3]:
# Load PDF
loader = PyPDFLoader(pdf_path)
# pages = loader.load()[2:] # Skip first two pages as before
pages = loader.load()
full_text = "\n".join(page.page_content for page in pages)

# Text Splitter (refined for graph structure)
# We'll primarily focus on splitting by Article for node creation
article_pattern_precise = re.compile(r'(제\d+조(?:의\d+)?\s*\(.+?\))\s*') # More precise pattern to capture article header
separators = [
    r"(제\d+조(?:의\d+)?\s*\(.+?\))", # 조 (Article) - Primary separator
    r"(제\d+장 [^\\n]+)",             # 장 (Chapter)
    r"(제\d+편 [^\\n]+)",             # 편 (Edition)
    "\n\n",                         # Paragraph
    "\n",                           # Line break
    " ",                            # Space
]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # Adjust as needed
    chunk_overlap=150, # Adjust as needed
    length_function=len,
    separators=separators,
    is_separator_regex=True,
)

raw_chunks = text_splitter.split_text(full_text)

# Process chunks to associate content with articles
articles = {}
current_article_id = None
current_content = ""

for chunk in raw_chunks:
    match = article_pattern_precise.match(chunk)
    if match:
        # If we encounter a new article, save the previous one (if any)
        if current_article_id:
            articles[current_article_id] = current_content.strip()

        # Start the new article
        current_article_id = match.group(1).strip()
        # Remove the matched article header from the beginning of the chunk content
        current_content = article_pattern_precise.sub("", chunk, count=1)
    else:
        # Append chunk content to the current article
        if current_article_id: # Only append if we are already inside an article
             current_content += "\n" + chunk

# Save the last processed article
if current_article_id:
    articles[current_article_id] = current_content.strip()


print(f"Processed {len(articles)} articles from PDF.")
# Example: Print the first processed article
if articles:
    first_article_id = list(articles.keys())[0]
    print(f"\n--- Example Article: {first_article_id} ---")
    print(articles[first_article_id][:500] + "...")

# Convert to Document objects for potential later use or consistency
article_docs = [Document(page_content=content, metadata={"article_id": article_id}) for article_id, content in articles.items()]

Processed 71 articles from PDF.

--- Example Article: 제4조(국외에 있는 내국선박 등에서 외국인이 범한 죄) ---
본법은 대한민국영역 외에 있는 대한민
국의 선박 또는 항공기 내에서 죄를 범한 외국인에게 적용한다.
 
제5조(외국인의 국외범)제5조(외국인의 국외범) 본법은 대한민국영역 외에서 다음에 기재한 죄를 범한 외국인에게 적용
한다.
1. 내란의 죄
2. 외환의 죄
3. 국기에 관한 죄
4. 통화에 관한 죄
5. 유가증권, 우표와 인지에 관한 죄
6. 문서에 관한 죄 중 제225조 내지 제230조
7. 인장에 관한 죄 중 제238조
 
제6조(대한민국과 대한민국 국민에 대한 국외범)제6조(대한민국과 대한민국 국민에 대한 국외범) 본법은 대한민국영역 외에서 대한민국 또는 대
한민국 국민에 대하여 전조에 기재한 이외의 죄를 범한 외국인에게 적용한다. 단 행위지의 법
률에 의하여 범죄를 구성하지 아니하거나 소추 또는 형의 집행을 면제할 경우에는 예외로 한
다.
 
형법
[시행 2010.10.16] [법률 제10259호, 2010.4.15, 일부개정]
법제처                      ...


In [4]:
# Load precedent JSON files
precedents = []
for filename in os.listdir(precedent_dir):
    if filename.endswith(".json"):
        filepath = os.path.join(precedent_dir, filename)
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Extract relevant fields
                precedent_info = {
                    "case_id": data.get("info", {}).get("caseNoID", filename.replace(".json", "")),
                    "case_name": data.get("info", {}).get("caseNm"),
                    "judgment_summary": data.get("jdgmn"),
                    "full_summary": " ".join([s.get("summ_contxt", "") for s in data.get("Summary", [])]),
                    "keywords": [kw.get("keyword") for kw in data.get("keyword_tagg", []) if kw.get("keyword")],
                    "referenced_rules": data.get("Reference_info", {}).get("reference_rules", "").split(',') if data.get("Reference_info", {}).get("reference_rules") else [],
                    "referenced_cases": data.get("Reference_info", {}).get("reference_court_case", "").split(',') if data.get("Reference_info", {}).get("reference_court_case") else [],
                }
                # Clean up referenced rules (extract article numbers)
                cleaned_rules = []
                rule_pattern = re.compile(r'제\d+조(?:의\d+)?') # Pattern to find "제X조" or "제X조의Y"
                for rule in precedent_info["referenced_rules"]:
                    # Find all matches in the rule string
                    matches = rule_pattern.findall(rule.strip())
                    cleaned_rules.extend(matches)
                precedent_info["referenced_rules"] = list(set(cleaned_rules)) # Keep unique article numbers

                precedents.append(precedent_info)
        except json.JSONDecodeError:
            print(f"Warning: Could not decode JSON from {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")


print(f"Loaded {len(precedents)} precedents.")
# Example: Print the first loaded precedent
if precedents:
    print("\n--- Example Precedent ---")
    print(json.dumps(precedents[0], indent=2, ensure_ascii=False))

Loaded 5404 precedents.

--- Example Precedent ---
{
  "case_id": "88도2209",
  "case_name": "매장및묘지등에관한법률위반, 사문서위조, 동행사, 조세범처벌법위반, 특정범죄가중처벌등에관한법률위반",
  "judgment_summary": "가. 작성명의자의 인영이나 주민등록번호의 등재가 누락된 문서가 사문서위조죄의 객체인 사문서에 해당하는지 여부\n나. 사문서위조 및 동행사죄가 조세범처벌법 제9조 소정의 조세포탈의 수단으로 행해진 경우 후자의 죄에 흡수되는지 여부(소극)",
  "full_summary": "사문서의 작성명의자의 인장이 압날되지 아니하고 주민등록번호가 기재되지 않았다고 하더라도, 일반인으로 하여금 그 작상명의자가 진정하게 작성한 사문서로 믿기에 충분할 정도의 형식과 외관을 갖추었으면 사문서위조죄 및 동행사죄의 객체가 되는 사문서라고 보아야 할 것이고, 사문서위조 및 동행사죄가 조세범처벌법 제9조 제1항 소정의 “사기 기타 부정한 행위로써 조세를 포탈”하기 위한 수단으로 행하여졌다고 하여 조세범처벌법 제9조 소정의 조세포탈죄에 흡수된다고 볼 수도 없는 것이므로, 논지는 이유가 없다.",
  "keywords": [
    "사문서위조",
    "동행사"
  ],
  "referenced_rules": [
    "제231조",
    "제9조",
    "제37조",
    "제234조"
  ],
  "referenced_cases": []
}


In [5]:
# 로드된 판례 중 무작위로 1,000개만 선택
import random
random.seed(42)  # 재현성을 위한 시드 설정 (선택사항)

# 전체 판례 수 저장
total_precedents = len(precedents)

# 무작위로 1,000개 선택 (또는 전체 판례 수가 1,000개보다 적다면 모두 선택)
sample_size = min(1000, total_precedents)
precedents = random.sample(precedents, sample_size)

print(f"Randomly selected {len(precedents)} precedents out of {total_precedents} total precedents.")

Randomly selected 1000 precedents out of 5404 total precedents.


In [6]:
# Connect to Neo4j
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    driver.verify_connectivity()
    print("Successfully connected to Neo4j.")
except Exception as e:
    print(f"Failed to connect to Neo4j: {e}")
    # Stop execution if connection fails
    raise

# Create constraints and indexes for faster lookups and embedding search
def setup_neo4j(driver, dimension):
    with driver.session(database="neo4j") as session:
        # Constraints for uniqueness
        session.run("CREATE CONSTRAINT article_id IF NOT EXISTS FOR (a:Article) REQUIRE a.id IS UNIQUE")
        session.run("CREATE CONSTRAINT precedent_id IF NOT EXISTS FOR (p:Precedent) REQUIRE p.id IS UNIQUE")
        session.run("CREATE CONSTRAINT keyword_text IF NOT EXISTS FOR (k:Keyword) REQUIRE k.text IS UNIQUE")

        # Vector index for Articles
        try:
            session.run(
                "CREATE VECTOR INDEX article_embedding IF NOT EXISTS "
                "FOR (a:Article) ON (a.embedding) "
                f"OPTIONS {{indexConfig: {{`vector.dimensions`: {dimension}, `vector.similarity_function`: 'cosine'}}}}"
            )
            print("Article vector index created or already exists.")
        except Exception as e:
            print(f"Error creating Article vector index (may require Neo4j 5.11+ and APOC): {e}")
            print("Continuing without vector index creation for Article.")


        # Vector index for Precedents
        try:
            session.run(
                "CREATE VECTOR INDEX precedent_embedding IF NOT EXISTS "
                "FOR (p:Precedent) ON (p.embedding) "
                f"OPTIONS {{indexConfig: {{`vector.dimensions`: {dimension}, `vector.similarity_function`: 'cosine'}}}}"
            )
            print("Precedent vector index created or already exists.")
        except Exception as e:
            print(f"Error creating Precedent vector index (may require Neo4j 5.11+ and APOC): {e}")
            print("Continuing without vector index creation for Precedent.")

        # Wait for indexes to come online (important!)
        print("Waiting for indexes to populate...")
        session.run("CALL db.awaitIndexes(300)") # Wait up to 300 seconds
        print("Indexes should be online.")


setup_neo4j(driver, embedding_dimension)

Successfully connected to Neo4j.
Article vector index created or already exists.
Precedent vector index created or already exists.
Waiting for indexes to populate...
Indexes should be online.


In [7]:
# Create Article nodes and generate/store embeddings
def create_article_nodes(driver, articles_dict, embed_model):
    print(f"Creating {len(articles_dict)} Article nodes...")
    count = 0
    start_time = time.time()
    with driver.session(database="neo4j") as session:
        for article_id, content in articles_dict.items():
            if not content: # Skip empty content
                print(f"Skipping article {article_id} due to empty content.")
                continue
            try:
                # Generate embedding
                embedding = embed_model.embed_query(content)

                # Create node in Neo4j
                session.run(
                    """
                    MERGE (a:Article {id: $article_id})
                    SET a.text = $content,
                        a.embedding = $embedding
                    """,
                    article_id=article_id,
                    content=content,
                    embedding=embedding
                )
                count += 1
                if count % 50 == 0:
                    print(f"  Processed {count}/{len(articles_dict)} articles...")
            except Exception as e:
                print(f"Error processing article {article_id}: {e}")

    end_time = time.time()
    print(f"Finished creating {count} Article nodes in {end_time - start_time:.2f} seconds.")

create_article_nodes(driver, articles, embedding_model)

Creating 71 Article nodes...
  Processed 50/71 articles...
Finished creating 71 Article nodes in 42.63 seconds.


In [8]:
# Create Precedent nodes, Keyword nodes, and relationships
def create_precedent_nodes_and_relationships(driver, precedents_list, embed_model):
    print(f"Creating {len(precedents_list)} Precedent nodes and relationships...")
    count = 0
    start_time = time.time()
    with driver.session(database="neo4j") as session:
        for precedent in precedents_list:
            # Use full_summary for embedding, or judgment_summary if full is empty
            text_to_embed = precedent.get("full_summary") or precedent.get("judgment_summary")
            if not text_to_embed:
                print(f"Skipping precedent {precedent.get('case_id')} due to empty summary.")
                continue

            try:
                # Generate embedding
                embedding = embed_model.embed_query(text_to_embed)

                # Create Precedent node
                session.run(
                    """
                    MERGE (p:Precedent {id: $case_id})
                    SET p.name = $case_name,
                        p.judgment_summary = $judgment_summary,
                        p.full_summary = $full_summary,
                        p.embedding = $embedding
                    """,
                    case_id=precedent["case_id"],
                    case_name=precedent["case_name"],
                    judgment_summary=precedent["judgment_summary"],
                    full_summary=precedent["full_summary"],
                    embedding=embedding
                )

                # Create Keyword nodes and relationships
                for keyword_text in precedent["keywords"]:
                    session.run(
                        """
                        MERGE (k:Keyword {text: $keyword_text})
                        WITH k
                        MATCH (p:Precedent {id: $case_id})
                        MERGE (p)-[:HAS_KEYWORD]->(k)
                        """,
                        keyword_text=keyword_text,
                        case_id=precedent["case_id"]
                    )

                # Create relationships to referenced Articles
                # Note: This uses the cleaned article IDs extracted earlier
                # It tries to match based on the "제X조" format.
                for article_id_ref in precedent["referenced_rules"]:
                     # Find Article nodes that START WITH the referenced ID (e.g., "제21조" should match "제21조(정당방위)")
                     # This is less precise but necessary if the exact title isn't in the reference.
                    session.run(
                        """
                        MATCH (p:Precedent {id: $case_id})
                        MATCH (a:Article)
                        WHERE a.id STARTS WITH $article_id_ref
                        MERGE (p)-[:REFERENCES_ARTICLE]->(a)
                        """,
                        case_id=precedent["case_id"],
                        article_id_ref=article_id_ref # Use the extracted "제X조"
                    )

                # Potential: Create relationships to other referenced Precedents (if needed)
                # for ref_case_id in precedent["referenced_cases"]:
                #    session.run(...) # MERGE (p)-[:REFERENCES_CASE]->(other_p:Precedent {id: ref_case_id})

                count += 1
                if count % 100 == 0:
                    print(f"  Processed {count}/{len(precedents_list)} precedents...")

            except Exception as e:
                print(f"Error processing precedent {precedent.get('case_id')}: {e}")

    end_time = time.time()
    print(f"Finished creating {count} Precedent nodes and relationships in {end_time - start_time:.2f} seconds.")


create_precedent_nodes_and_relationships(driver, precedents, embedding_model)

# Close the driver connection when done
# driver.close() # Keep it open for querying in the next step

Creating 1000 Precedent nodes and relationships...
  Processed 100/1000 precedents...
  Processed 200/1000 precedents...
  Processed 300/1000 precedents...
  Processed 400/1000 precedents...
  Processed 500/1000 precedents...
  Processed 600/1000 precedents...
  Processed 700/1000 precedents...
  Processed 800/1000 precedents...
  Processed 900/1000 precedents...
  Processed 1000/1000 precedents...
Finished creating 1000 Precedent nodes and relationships in 1177.54 seconds.


In [9]:
# Example RAG query using vector similarity search
def query_graph_rag(driver, query_text, embed_model, top_k=3):
    print(f"\n--- Querying RAG for: '{query_text}' ---")
    start_time = time.time()

    # Embed the query
    query_embedding = embed_model.embed_query(query_text)

    results = []
    with driver.session(database="neo4j") as session:
        # Find similar Articles
        try:
            article_res = session.run(
                """
                CALL db.index.vector.queryNodes('article_embedding', $top_k, $query_embedding) YIELD node, score
                RETURN node.id AS article_id, node.text AS text, score
                """,
                top_k=top_k,
                query_embedding=query_embedding
            )
            for record in article_res:
                 results.append({
                     "type": "Article",
                     "id": record["article_id"],
                     "score": record["score"],
                     "text": record["text"][:300] + "..." # Preview
                 })
        except Exception as e:
            print(f"Could not query Article vector index: {e}")


        # Find similar Precedents
        try:
            precedent_res = session.run(
                """
                CALL db.index.vector.queryNodes('precedent_embedding', $top_k, $query_embedding) YIELD node, score
                MATCH (node)-[:REFERENCES_ARTICLE]->(a:Article)
                OPTIONAL MATCH (node)-[:HAS_KEYWORD]->(k:Keyword)
                RETURN node.id AS case_id, node.name as case_name, node.full_summary AS text, score,
                       collect(DISTINCT a.id) as referenced_articles,
                       collect(DISTINCT k.text) as keywords
                """,
                top_k=top_k,
                query_embedding=query_embedding
            )
            for record in precedent_res:
                 results.append({
                     "type": "Precedent",
                     "id": record["case_id"],
                     "name": record["case_name"],
                     "score": record["score"],
                     "text": record["text"][:300] + "...", # Preview
                     "referenced_articles": record["referenced_articles"],
                     "keywords": record["keywords"]
                 })
        except Exception as e:
            print(f"Could not query Precedent vector index: {e}")


    end_time = time.time()
    print(f"Query completed in {end_time - start_time:.2f} seconds.")

    # Sort results by score (descending) and take top K overall
    results.sort(key=lambda x: x["score"], reverse=True)

    print("\n--- Top Results ---")
    for i, res in enumerate(results[:top_k]):
        print(f"{i+1}. Type: {res['type']}, ID: {res['id']}, Score: {res['score']:.4f}")
        if res['type'] == 'Precedent':
            print(f"   Name: {res.get('name')}")
            print(f"   Keywords: {res.get('keywords')}")
            print(f"   Referenced Articles: {res.get('referenced_articles')}")
        print(f"   Text Preview: {res['text']}")
        print("-" * 20)

    return results[:top_k] # Return top K overall results


# Test query
query = "정당방위의 요건은 무엇인가?"
retrieved_context = query_graph_rag(driver, query, embedding_model, top_k=3)

# Close driver when completely finished
driver.close()
print("\nNeo4j driver closed.")


--- Querying RAG for: '정당방위의 요건은 무엇인가?' ---
Query completed in 1.46 seconds.

--- Top Results ---
1. Type: Precedent, ID: 2009도2114, Score: 0.6935
   Name: 특수공무집행방해[변경된죄명:폭력행위등처벌에관한법률위반(공동폭행)]
   Keywords: ['사회상규에 위배', '소극적인 방어행위']
   Referenced Articles: ['제260조(폭행, 존속폭행)']
   Text Preview: 비록 경찰관들의 위법한 상경 제지 행위에 대항하기 위하여 한 것이라 하더라도, 피고인들이 다른 시위참가자들과 공동하여 위와 같이 경찰관들을 때리고 진압방패와 채증장비를 빼앗는 등의 폭행행위를 한 것은 소극적인 방어행위를 넘어서 공격의 의사를 포함하여 이루어진 것으로서 그 수단과 방법에 있어서 상당성이 인정된다고 보기 어려우며 긴급하고 불가피한 수단이었다고 볼 수도 없으므로, 이를 사회상규에 위배되지 아니하는 정당행위나 현재의 부당한 침해를 방어하기 위한 정당방위에 해당한다고 볼 수 없다.
그럼에도 불구하고, 원심은 그 판시와 같은 ...
--------------------
2. Type: Precedent, ID: 83누383, Score: 0.6789
   Name: 영업정지처분무효확인
   Keywords: ['법령 위반행위']
   Referenced Articles: ['제1조 (시행일)']
   Text Preview: 정당한 절차에 의하지 않고 구두에 의한 하도급계약을 체결하여 공사를 시작한 때에건설업법 제34조 제3항의 위반행위를 범한 것이 되니 그 위반행위를 이유로 한 행정상의 제재처분(행위당시에는 필요적 취소사유)을 하려면 그 위반행위 이후 법령의 변경에 의하여 처분의 종류를 달리(영업정지 사유로) 규정하였다 하더라도 그 법률적용에 관한특별한 규정이 없다면 위반행위 당시에 시행되던 법령을 근거로 처분을 하여야 마땅하다....
